# OCR on SageMaker--Build Image

## 0 运行环境
本文在boto3 1.17.17下测试通过。

In [ ]:
import boto3
print(boto3.__version__)

如果版本较低，可运行下行代码进行升级，升级完毕后，需要重启Kernal，然后再检查版本

In [ ]:
!pip install boto3 --upgrade -i https://opentuna.cn/pypi/web/simple/

## 1 准备PaddleOCR

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR dockersource

## 2 拷贝web相关文件到docker build目录

In [ ]:
!cp -r source/* dockersource

## 3 准备模型
如果有自己的模型，可以直接存放到`dockersource/inference`目录下，如果没有，可直接使用官方的模型

In [ ]:
%cd dockersource
!mkdir inference
%cd inference
# 下载超轻量级中文OCR模型的检测模型并解压
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_det_infer.tar && tar xf ch_ppocr_mobile_v2.0_det_infer.tar && rm ch_ppocr_mobile_v2.0_det_infer.tar
# 下载超轻量级中文OCR模型的识别模型并解压
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_rec_infer.tar && tar xf ch_ppocr_mobile_v2.0_rec_infer.tar && rm ch_ppocr_mobile_v2.0_rec_infer.tar
# 下载超轻量级中文OCR模型的文本方向分类器模型并解压
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar && tar xf ch_ppocr_mobile_v2.0_cls_infer.tar && rm ch_ppocr_mobile_v2.0_cls_infer.tar
%cd ../..

## 4 设置相关名称

In [ ]:
ecr_repository = 'ocr-inference'
##前面为paddlepaddle版本，后面为paddleocr版本
tag = '2.0.2-2.0-cpu'

## 5 Build image
CPU推理使用Dockerfile，GPU推理使用Dockerfile.gpu

In [ ]:
%%time
!docker build -t $ecr_repository:$tag -f Dockerfile .

## 6 本地推理(可选)

### 6.1 启动服务

CPU推理

In [ ]:
!docker run -p 8080:8080 -d --rm $ecr_repository:$tag serve

GPU推理

In [ ]:
!nvidia-docker run -p 8080:8080 -d --rm $ecr_repository:$tag serve

### 6.2 发送请求

直接发送图片

In [ ]:
#自行修改文件路径
!curl --data-binary @../image/10.jpg -H "Content-Type:image/jpeg" -X POST http://127.0.0.1:8080/invocations

从S3获取图片

In [ ]:
#自行修改data内容
!curl --data '{"bucket":"nowfox","image_uri":["data/test/008.jpg"]}' -H "Content-Type: application/json" -X POST http://127.0.0.1:8080/invocations

## 7 把image push到ECR

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository

In [ ]:
import boto3
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
image_uri = '{}.dkr.ecr.{}.amazonaws.com.cn/{}'.format(account_id, region, ecr_repository + ":" + tag)
!docker tag $ecr_repository:$tag $image_uri
!$(aws ecr get-login --no-include-email)
!docker push $image_uri